# CMSC 461 Project Notebook

**Name:** Robert Rose  
**Semester:** Spring 2019

User Guide Note: To run a cell, hit shift and enter.

First ensure that the connector is installed via pip:

In [1]:
!pip install mysql-connector-python

You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import mysql.connector
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Get database password:

In [3]:
import getpass
db_user = 'root'
db_password = getpass.getpass()

········


In [5]:
config = {
    'user': db_user,
    'password': db_password,
    'host': 'localhost',
    'port': 3306,
    'database': 'project',
    'raise_on_warnings': False
}

In [7]:
db_connection = mysql.connector.connect(**config)

In [8]:
cur = db_connection.cursor()

## CRUD Operations

### Barcodes

Barcodes must be 13 characters long. There is no need for UPDATE commands on the barcode because there's only one value in the table.

In [19]:
@interact_manual(barcode='13 CHARACTERS')
def create_barcode(barcode):
    query = f"""
    INSERT INTO barcodes(barcode)
    VALUES('{barcode}')
    """
    if len(barcode) == 13:
        try:
            cur.execute(query)
            print(f"Barcode {barcode} successfully inserted!")
            db_connection.commit()
        except Exception as err:
            print(f"An error {err} occurred")
            db_connection.rollback()
    else:
        print("Barcode must be 13 characters long.")

interactive(children=(Text(value='13 CHARACTERS', description='barcode'), Button(description='Run Interact', s…

In [29]:
@interact(barcode='')
def search_barcode(barcode):
    # This searches to see if the barcode is in the database and what ids it corresponds to.
    query = """
    SELECT * FROM barcode_lookup_view
    WHERE barcode LIKE CONCAT (%s, '%')
    """
    print("('Barcode', 'Species', 'Cultivar', 'tray_id', 'pots_id', 'station_id')")
    cur.execute(query, (barcode,))
    results = cur.fetchall()
    for res in results:
        print(res)

interactive(children=(Text(value='', description='barcode'), Output()), _dom_classes=('widget-interact',))